# Transformers:

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-03-03 07:16:59--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-03-03 07:16:59 (18.5 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open('input.txt', 'r') as file:
  text = file.read()

print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [ ]:
# All possible characters the model can see and emmit.

vocabulary = sorted(list(set(text)))
vocabulary

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [ ]:
vocab_size = len(vocabulary)
vocab_size

65

In [ ]:
stoi = {s: i for i, s in enumerate(vocabulary)}
itos = {i: s for i, s in enumerate(vocabulary)}

stoi, itos

({'\n': 0,
  ' ': 1,
  '!': 2,
  '$': 3,
  '&': 4,
  "'": 5,
  ',': 6,
  '-': 7,
  '.': 8,
  '3': 9,
  ':': 10,
  ';': 11,
  '?': 12,
  'A': 13,
  'B': 14,
  'C': 15,
  'D': 16,
  'E': 17,
  'F': 18,
  'G': 19,
  'H': 20,
  'I': 21,
  'J': 22,
  'K': 23,
  'L': 24,
  'M': 25,
  'N': 26,
  'O': 27,
  'P': 28,
  'Q': 29,
  'R': 30,
  'S': 31,
  'T': 32,
  'U': 33,
  'V': 34,
  'W': 35,
  'X': 36,
  'Y': 37,
  'Z': 38,
  'a': 39,
  'b': 40,
  'c': 41,
  'd': 42,
  'e': 43,
  'f': 44,
  'g': 45,
  'h': 46,
  'i': 47,
  'j': 48,
  'k': 49,
  'l': 50,
  'm': 51,
  'n': 52,
  'o': 53,
  'p': 54,
  'q': 55,
  'r': 56,
  's': 57,
  't': 58,
  'u': 59,
  'v': 60,
  'w': 61,
  'x': 62,
  'y': 63,
  'z': 64},
 {0: '\n',
  1: ' ',
  2: '!',
  3: '$',
  4: '&',
  5: "'",
  6: ',',
  7: '-',
  8: '.',
  9: '3',
  10: ':',
  11: ';',
  12: '?',
  13: 'A',
  14: 'B',
  15: 'C',
  16: 'D',
  17: 'E',
  18: 'F',
  19: 'G',
  20: 'H',
  21: 'I',
  22: 'J',
  23: 'K',
  24: 'L',
  25: 'M',
  26: 'N',
  27:

In [ ]:
encode = lambda s: [stoi[s] for s in list(s)]
encode('Hi There!')

[20, 47, 1, 32, 46, 43, 56, 43, 2]

In [ ]:
decode = lambda l: ''.join([itos[i] for i in l])
decode([20, 47, 1, 32, 46, 43, 56, 43, 2])

'Hi There!'

In [ ]:
import torch # ML Framework.

In [ ]:
data = torch.tensor(encode(text))
data

tensor([18, 47, 56,  ..., 45,  8,  0])

In [ ]:
len(data), len(text)

(1115394, 1115394)

In [ ]:
train_size = int(0.9 * len(data))
train_data = data[:train_size]
test_data = data[train_size:]
len(train_data), len(test_data), len(train_data) + len(test_data)

(1003854, 111540, 1115394)

In [ ]:
block_size = 8 # Context Window. (Maximum number of tokens the model can process at a time/ uske baad truncation).
train_data[:block_size + 1] # Block size + 1 tokens me Block size possible combinations hai.

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
torch.randint(0, 5, (5, ))

tensor([0, 1, 2, 3, 0])

In [ ]:
torch.manual_seed(1337)

batch_size = 4 # Batches of inputs to the Neural Network / Batches of input sequence/tokens.
block_size = 8

def get_data(data_type):
  _data = train_data if data_type == 'train' else test_data
  idxs = torch.randint(0, len(_data) - block_size, (batch_size, )) # Batch size number of random indeces.
  xb = torch.stack([_data[idx: idx + block_size] for idx in idxs], dim = 0)
  yb = torch.stack([_data[idx + 1: idx + block_size + 1] for idx in idxs], dim = 0)
  return xb, yb

In [ ]:
# Har batch (4) me block size (8) examples packed hai. Total: 32 sequences of text.

xb, yb = get_data('train')
xb, yb

(tensor([[24, 43, 58,  5, 57,  1, 46, 43],
         [44, 53, 56,  1, 58, 46, 39, 58],
         [52, 58,  1, 58, 46, 39, 58,  1],
         [25, 17, 27, 10,  0, 21,  1, 54]]),
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
         [53, 56,  1, 58, 46, 39, 58,  1],
         [58,  1, 58, 46, 39, 58,  1, 46],
         [17, 27, 10,  0, 21,  1, 54, 39]]))

## Basic Bi-Gram model:

In [ ]:
# Upar humne one hot encoding ki thi. Yaha hum embeddings use karenge.
# Kind of a vector representation of my discrete/distinct tokens (65)/ vocabulary.

import torch.nn as nn
import torch.nn.functional as F

In [ ]:
embedding_layer = nn.Embedding(5, 3) # 5 embeddings of size 3. Possible options/indeces -> 0, 1, 2, 3, 4
embedding_layer # You get a vector for all your possible tokens. Initially these numbers/embeddings are random/->updated (with training).

Embedding(5, 3)

In [ ]:
embedding_layer(torch.tensor(0)) # 0D -> 1D

tensor([-1.2054, -0.9122, -1.2502], grad_fn=<EmbeddingBackward0>)

In [ ]:
embedding_layer(torch.tensor([0, 1])) # 1D -> 2D (Kind of an output that can simpley be fed into a NN)

tensor([[-1.2054, -0.9122, -1.2502],
        [ 0.8032, -0.2071,  0.0544]], grad_fn=<EmbeddingBackward0>)

In [ ]:
embedding_layer(torch.tensor([[0, 1], [2, 3]])) # 2D -> 3D

tensor([[[-1.2054, -0.9122, -1.2502],
         [ 0.8032, -0.2071,  0.0544]],

        [[ 0.1378, -0.3889,  0.5133],
         [ 0.3319,  0.6300,  0.5815]]], grad_fn=<EmbeddingBackward0>)

In [ ]:
embedding_layer(torch.tensor([[0, 1], [2, 3]])).shape

torch.Size([2, 2, 3])

In [ ]:
xb.shape, xb # (B, T)

(torch.Size([4, 8]),
 tensor([[24, 43, 58,  5, 57,  1, 46, 43],
         [44, 53, 56,  1, 58, 46, 39, 58],
         [52, 58,  1, 58, 46, 39, 58,  1],
         [25, 17, 27, 10,  0, 21,  1, 54]]))

In [ ]:
nn.Embedding(vocab_size, vocab_size)(xb).shape # 1 embedding for each token in the vocabulary.
# A 65-sized vector for all the 4 X 8 = 32 tokens. # B, T, C

torch.Size([4, 8, 65])

In [ ]:
help(F.cross_entropy) # - log likelihood.

Help on function cross_entropy in module torch.nn.functional:

cross_entropy(input: torch.Tensor, target: torch.Tensor, weight: Optional[torch.Tensor] = None, size_average: Optional[bool] = None, ignore_index: int = -100, reduce: Optional[bool] = None, reduction: str = 'mean', label_smoothing: float = 0.0) -> torch.Tensor
    Compute the cross entropy loss between input logits and target.
    
    See :class:`~torch.nn.CrossEntropyLoss` for details.
    
    Args:
        input (Tensor) : Predicted unnormalized logits;
            see Shape section below for supported shapes.
        target (Tensor) : Ground truth class indices or class probabilities;
            see Shape section below for supported shapes.
        weight (Tensor, optional): a manual rescaling weight given to each
            class. If given, has to be a Tensor of size `C`
        size_average (bool, optional): Deprecated (see :attr:`reduction`). By default,
            the losses are averaged over each loss element i

In [ ]:
# Predicted unnormalized logits & Ground truth class indeces.
logits = nn.Embedding(vocab_size, vocab_size)(xb)
B, T, C = logits.shape
logits.view(B * T, C) # Embeddings for all 32 tokens. # Unnormalized logits.

tensor([[ 1.2815,  0.4047,  0.0632,  ..., -0.2625,  1.1260, -0.1366],
        [-0.5828,  1.0230, -1.1334,  ...,  0.1380, -0.0623,  0.4172],
        [-1.3142, -0.2682, -0.1609,  ...,  0.9439, -2.1673,  2.0899],
        ...,
        [-0.7827,  2.4638,  0.7175,  ..., -0.2406,  0.5494,  0.0308],
        [-0.1004,  0.2456,  0.0935,  ...,  0.0534, -0.1949,  0.8603],
        [ 1.3676,  0.1808,  0.5740,  ...,  0.5384,  0.2382, -1.5247]],
       grad_fn=<ViewBackward0>)

In [ ]:
# Output tokens -> yb
yb.view(-1, )

tensor([43, 58,  5, 57,  1, 46, 43, 39, 53, 56,  1, 58, 46, 39, 58,  1, 58,  1,
        58, 46, 39, 58,  1, 46, 17, 27, 10,  0, 21,  1, 54, 39])

In [ ]:
F.cross_entropy(logits.view(B * T, C), yb.view(-1, ))

tensor(4.7724, grad_fn=<NllLossBackward0>)

In [ ]:
logits = logits.view(B * T, C)
counts = logits.exp()
probabilities = counts / counts.sum(1, True)
-probabilities[range(len(logits)), yb.view(-1, )].log().mean() # Exactly the same as negative log likelihood that we calculated before.

tensor(4.7724, grad_fn=<NegBackward0>)

In [ ]:
# - sigma(pi * log(pi)) ??
-(probabilities * probabilities.log()).mean()

tensor(0.0576, grad_fn=<NegBackward0>)

## Class implementation of NN:

In [ ]:
class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # Channel Dimension (vocab_size).

  def forward(self, x, y = None):
    logits = self.token_embedding_table(x) # log counts. (B, T, C)

    if y is None:
      loss = None
    else:
      B, T, C = logits.shape
      loss = F.cross_entropy(logits.view(B * T, C), y.view(B * T))

    return logits, loss

  def generate(self, x, max_tokens):
    for _ in range(max_tokens):
      logits, loss = self(x) # Pura x pass kia. (B, T) -> (B, T, C)

      logits = logits[:, -1, :] # Per batch (Last token ki embeddings le li) -> 65 (B * C) (Last token in time.)
      probabilities = F.softmax(logits, dim = 1) # Auto-built function for us. to do e^x/sigma(e^x) (B * C)

      x_next = torch.multinomial(probabilities, 1, replacement = True) # (B, 1)

      x = torch.concat([x, x_next], dim = 1) # (B, T + 1)

    return x

In [ ]:
model = BigramLanguageModel()
model(xb)

(tensor([[[ 0.3498,  1.3103, -0.9592,  ...,  1.0153, -0.1428,  2.3518],
          [ 2.2402,  0.6659,  0.8235,  ..., -1.1192,  2.5383, -0.4066],
          [-1.6217, -1.7992, -1.4361,  ..., -0.4915,  0.7770,  0.9001],
          ...,
          [-0.5502, -0.2549,  0.7017,  ...,  0.1585, -1.6419, -1.5263],
          [ 0.3879, -1.0859,  1.5126,  ...,  0.3447,  0.8484, -0.2543],
          [ 2.2402,  0.6659,  0.8235,  ..., -1.1192,  2.5383, -0.4066]],
 
         [[-0.2413,  1.0492, -0.5896,  ..., -0.3442,  0.1582, -0.1497],
          [-1.6302,  1.4898, -0.4206,  ..., -1.9137, -0.9732, -0.2703],
          [-1.4160,  0.2531,  1.1756,  ..., -0.1245,  0.0146,  1.8470],
          ...,
          [ 0.3879, -1.0859,  1.5126,  ...,  0.3447,  0.8484, -0.2543],
          [-0.2533,  0.5686,  0.3427,  ...,  0.7236,  1.4675,  1.3112],
          [-1.6217, -1.7992, -1.4361,  ..., -0.4915,  0.7770,  0.9001]],
 
         [[-0.0115, -0.3571,  0.0865,  ...,  1.6329, -0.1404, -0.7568],
          [-1.6217, -1.7992,

In [ ]:
model(xb, yb)

(tensor([[[ 0.3498,  1.3103, -0.9592,  ...,  1.0153, -0.1428,  2.3518],
          [ 2.2402,  0.6659,  0.8235,  ..., -1.1192,  2.5383, -0.4066],
          [-1.6217, -1.7992, -1.4361,  ..., -0.4915,  0.7770,  0.9001],
          ...,
          [-0.5502, -0.2549,  0.7017,  ...,  0.1585, -1.6419, -1.5263],
          [ 0.3879, -1.0859,  1.5126,  ...,  0.3447,  0.8484, -0.2543],
          [ 2.2402,  0.6659,  0.8235,  ..., -1.1192,  2.5383, -0.4066]],
 
         [[-0.2413,  1.0492, -0.5896,  ..., -0.3442,  0.1582, -0.1497],
          [-1.6302,  1.4898, -0.4206,  ..., -1.9137, -0.9732, -0.2703],
          [-1.4160,  0.2531,  1.1756,  ..., -0.1245,  0.0146,  1.8470],
          ...,
          [ 0.3879, -1.0859,  1.5126,  ...,  0.3447,  0.8484, -0.2543],
          [-0.2533,  0.5686,  0.3427,  ...,  0.7236,  1.4675,  1.3112],
          [-1.6217, -1.7992, -1.4361,  ..., -0.4915,  0.7770,  0.9001]],
 
         [[-0.0115, -0.3571,  0.0865,  ...,  1.6329, -0.1404, -0.7568],
          [-1.6217, -1.7992,

In [ ]:
print(decode(model.generate(torch.tensor([[0]]), 100)[0].tolist()))


ViYMNUlzu'-hXWPRIRiNQuF
UyEiysBx3KtjcOOCeOhUlM.pq:RTT;jdQuF:YEHrM!m;UmCzoFAxHtHu;&p&
j$ bErVaOQu'WvX


In [ ]:
# Training process.

model.parameters() # Use/Pass all the model parameters to the optimizer object.

<generator object Module.parameters at 0x795c00135ee0>

In [ ]:
list(model.parameters()), list(model.parameters())[0].shape # vocab_size, vocab_size weights for an embedding layer of this dimension.

([Parameter containing:
  tensor([[ 1.1891,  0.4247,  1.4639,  ...,  0.8044, -1.9819,  1.0756],
          [-0.5502, -0.2549,  0.7017,  ...,  0.1585, -1.6419, -1.5263],
          [-0.0566,  0.9142, -0.2427,  ..., -0.1146, -1.3816, -0.8433],
          ...,
          [-0.0275, -0.8411, -0.5339,  ...,  0.0678,  0.6898, -0.2888],
          [ 0.2983,  0.0298,  2.1243,  ...,  1.4154,  0.2682, -2.1083],
          [ 0.6007, -0.8245,  0.1892,  ...,  0.5648,  0.1959, -0.5216]],
         requires_grad=True)],
 torch.Size([65, 65]))

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3)
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0.01
)

In [ ]:
# Training Loop.

batch_size = 32
for i in range(20000):
  xb, yb = get_data('train')
  logits, loss = model(xb, yb)

  optimizer.zero_grad(set_to_none = True)
  loss.backward()

  optimizer.step() # Update weights.

  print(loss.item())

Streaming output truncated to the last 5000 lines.
2.492692470550537
2.4917163848876953
2.470090627670288
2.3553788661956787
2.432141065597534
2.3853700160980225
2.5103354454040527
2.316120147705078
2.533888101577759
2.490656614303589
2.3366808891296387
2.50423002243042
2.532947540283203
2.4066920280456543
2.4915266036987305
2.4837071895599365
2.5582199096679688
2.538115978240967
2.4007411003112793
2.4214749336242676
2.3402693271636963
2.3532450199127197
2.4605202674865723
2.461646795272827
2.4856483936309814
2.5099072456359863
2.5644752979278564
2.435406446456909
2.3799827098846436
2.3693389892578125
2.3270442485809326
2.3448550701141357
2.451723575592041
2.5898542404174805
2.482839822769165
2.445624351501465
2.5033938884735107
2.4439496994018555
2.482853412628174
2.492849111557007
2.4715545177459717
2.421292304992676
2.486783742904663
2.494849681854248
2.358307361602783
2.5199904441833496
2.3289263248443604
2.539226770401001
2.4005813598632812
2.470928907394409
2.3402035236358643
2.3

In [ ]:
print(decode(model.generate(torch.tensor([[0]]), 500)[0].tolist()))


thit RD:
Gom.
I INomere y ghesen cond
YCouchin, w?

Te counere ne ung;
GE n;
II t.

He verve o was.


Hes aift NTHEDIO:
Sl:
Whinke ngt?
MAms NGO shemo too mo anthatinthakes f utous as Agonteopr botherore thind spat PTEShiat ureraierio pr son me LO:
Wats me S:
To tllingewe lley ayom

Mo;
Latanssuromas:

Y:
PE:
Therucover, min ld te o e, un rd o s hthecals,

WI thank m:


NTharu t irlendoucin,
Y: Tisteriomad t.
Yor f.

S:
G, g w,

Hee:


NGLI,
JOreden t t IVo sl
LLTre!


APrs whes ge d f.

WI beco


## Attention.

In [ ]:
# Mathematical trick. (Attention me idea:)
# Currently we are using only the the last token in time to predict the next token/ Context window ka to scope aaya hi nahi hai.
# Attention -> Let the tokens within the context window, talk to each other.

# One good way -> in time T, if we represent each token as an average of all the tokens up until in the past. (Future tokens won't talk).

B, T, C = 4, 8, 2
x = torch.randn((B, T, C))
x

tensor([[[ 1.4752, -0.3331],
         [-2.0985, -0.9119],
         [ 0.4256,  2.2927],
         [ 1.3981,  0.7014],
         [ 0.5085, -0.2177],
         [ 0.1882,  1.4853],
         [ 1.6761, -1.1527],
         [ 0.3777, -0.7961]],

        [[-1.1295,  0.1419],
         [-1.9248, -1.6166],
         [ 0.4086, -0.2344],
         [ 0.4223,  0.2934],
         [ 1.6078, -2.2084],
         [ 1.5166, -0.3172],
         [ 0.4452,  1.5462],
         [ 0.1232, -1.8541]],

        [[-0.1258, -1.4535],
         [-0.5607, -0.6266],
         [-0.5190,  2.6530],
         [-0.5988,  0.6044],
         [ 0.4774,  0.9256],
         [-1.7807,  0.9799],
         [-0.9383,  0.0622],
         [-0.7525, -1.4755]],

        [[-1.0214, -0.2547],
         [ 0.5311, -0.4315],
         [ 0.7802, -1.1138],
         [ 0.9651,  0.6793],
         [ 0.7532,  0.3092],
         [-0.7913, -0.7759],
         [ 0.5681,  1.0482],
         [-0.2692,  0.3178]]])

In [ ]:
x[0] # 1 batch.

tensor([[ 1.4752, -0.3331],
        [-2.0985, -0.9119],
        [ 0.4256,  2.2927],
        [ 1.3981,  0.7014],
        [ 0.5085, -0.2177],
        [ 0.1882,  1.4853],
        [ 1.6761, -1.1527],
        [ 0.3777, -0.7961]])

In [ ]:
x[0][:2, :].mean(dim = 0)

tensor([-0.3116, -0.6225])

### Method 1 (Simplest)/Loop:

In [ ]:
# This avg representation is also referred to as BOW.
# We will se 3 different methods to calculate this average of all the tokens in the past.

xbow1 = torch.zeros((B, T, C))
xbow1

tensor([[[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]]])

In [ ]:
for b in range(B):
  for t in range(T):
    xbow1[b, t] = x[b][:t + 1, :].mean(dim = 0)

xbow1

tensor([[[ 1.4752, -0.3331],
         [-0.3116, -0.6225],
         [-0.0659,  0.3492],
         [ 0.3001,  0.4373],
         [ 0.3418,  0.3063],
         [ 0.3162,  0.5028],
         [ 0.5105,  0.2663],
         [ 0.4939,  0.1335]],

        [[-1.1295,  0.1419],
         [-1.5272, -0.7374],
         [-0.8819, -0.5697],
         [-0.5559, -0.3539],
         [-0.1231, -0.7248],
         [ 0.1501, -0.6569],
         [ 0.1923, -0.3422],
         [ 0.1837, -0.5312]],

        [[-0.1258, -1.4535],
         [-0.3433, -1.0400],
         [-0.4018,  0.1910],
         [-0.4511,  0.2944],
         [-0.2654,  0.4206],
         [-0.5179,  0.5138],
         [-0.5780,  0.4493],
         [-0.5998,  0.2087]],

        [[-1.0214, -0.2547],
         [-0.2451, -0.3431],
         [ 0.0966, -0.6000],
         [ 0.3138, -0.2802],
         [ 0.4017, -0.1623],
         [ 0.2028, -0.2646],
         [ 0.2550, -0.0770],
         [ 0.1895, -0.0277]]])

In [ ]:
x # But ye method kehte hai efficient nahi hai.

tensor([[[ 1.4752, -0.3331],
         [-2.0985, -0.9119],
         [ 0.4256,  2.2927],
         [ 1.3981,  0.7014],
         [ 0.5085, -0.2177],
         [ 0.1882,  1.4853],
         [ 1.6761, -1.1527],
         [ 0.3777, -0.7961]],

        [[-1.1295,  0.1419],
         [-1.9248, -1.6166],
         [ 0.4086, -0.2344],
         [ 0.4223,  0.2934],
         [ 1.6078, -2.2084],
         [ 1.5166, -0.3172],
         [ 0.4452,  1.5462],
         [ 0.1232, -1.8541]],

        [[-0.1258, -1.4535],
         [-0.5607, -0.6266],
         [-0.5190,  2.6530],
         [-0.5988,  0.6044],
         [ 0.4774,  0.9256],
         [-1.7807,  0.9799],
         [-0.9383,  0.0622],
         [-0.7525, -1.4755]],

        [[-1.0214, -0.2547],
         [ 0.5311, -0.4315],
         [ 0.7802, -1.1138],
         [ 0.9651,  0.6793],
         [ 0.7532,  0.3092],
         [-0.7913, -0.7759],
         [ 0.5681,  1.0482],
         [-0.2692,  0.3178]]])

### Method 2: Matrix multiplication:

In [ ]:
a = torch.ones((3, 3)) # Suppose ye Time dimension me hai (T, T)
a

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [ ]:
b = torch.randint(1, 5, (3, 2)) # Ye Time * Channel dimension hai (T, C)
b

tensor([[4, 2],
        [4, 2],
        [3, 4]])

In [ ]:
a.tril() # Lower triangular matrix.

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [ ]:
a.tril() @ b.float() # Sum uptill that token in the time. (Exactly what we want)

tensor([[ 4.,  2.],
        [ 8.,  4.],
        [11.,  8.]])

In [ ]:
a.tril().sum(dim = 1, keepdim = True) # & dividing by this will solve our purpose (of getting the average upto the ith token)

tensor([[1.],
        [2.],
        [3.]])

In [ ]:
a.tril() @ b.float() / a.tril().sum(1, True)

tensor([[4.0000, 2.0000],
        [4.0000, 2.0000],
        [3.6667, 2.6667]])

In [ ]:
a = torch.ones((T, T))
a

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [ ]:
a.tril()

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [ ]:
a.tril() @ x # (T, T) @ (B, T, C) -> Broadcast across all batches (Matching occurs from right).

tensor([[[ 1.4752, -0.3331],
         [-0.6233, -1.2450],
         [-0.1977,  1.0477],
         [ 1.2004,  1.7491],
         [ 1.7089,  1.5314],
         [ 1.8971,  3.0168],
         [ 3.5732,  1.8640],
         [ 3.9508,  1.0679]],

        [[-1.1295,  0.1419],
         [-3.0543, -1.4747],
         [-2.6458, -1.7092],
         [-2.2235, -1.4157],
         [-0.6157, -3.6241],
         [ 0.9008, -3.9414],
         [ 1.3461, -2.3952],
         [ 1.4693, -4.2493]],

        [[-0.1258, -1.4535],
         [-0.6865, -2.0800],
         [-1.2055,  0.5730],
         [-1.8043,  1.1774],
         [-1.3269,  2.1030],
         [-3.1077,  3.0829],
         [-4.0460,  3.1451],
         [-4.7985,  1.6696]],

        [[-1.0214, -0.2547],
         [-0.4902, -0.6862],
         [ 0.2899, -1.8000],
         [ 1.2550, -1.1208],
         [ 2.0083, -0.8116],
         [ 1.2169, -1.5875],
         [ 1.7850, -0.5393],
         [ 1.5158, -0.2215]]])

In [ ]:
xbow2 = (a.tril() @ x) / a.tril().sum(1, True)
xbow2

tensor([[[ 1.4752, -0.3331],
         [-0.3116, -0.6225],
         [-0.0659,  0.3492],
         [ 0.3001,  0.4373],
         [ 0.3418,  0.3063],
         [ 0.3162,  0.5028],
         [ 0.5105,  0.2663],
         [ 0.4939,  0.1335]],

        [[-1.1295,  0.1419],
         [-1.5272, -0.7374],
         [-0.8819, -0.5697],
         [-0.5559, -0.3539],
         [-0.1231, -0.7248],
         [ 0.1501, -0.6569],
         [ 0.1923, -0.3422],
         [ 0.1837, -0.5312]],

        [[-0.1258, -1.4535],
         [-0.3433, -1.0400],
         [-0.4018,  0.1910],
         [-0.4511,  0.2944],
         [-0.2654,  0.4206],
         [-0.5179,  0.5138],
         [-0.5780,  0.4493],
         [-0.5998,  0.2087]],

        [[-1.0214, -0.2547],
         [-0.2451, -0.3431],
         [ 0.0966, -0.6000],
         [ 0.3138, -0.2802],
         [ 0.4017, -0.1623],
         [ 0.2028, -0.2646],
         [ 0.2550, -0.0770],
         [ 0.1895, -0.0277]]])

In [ ]:
torch.allclose(xbow1, xbow2)

True

In [ ]:
# Or we can tell it something like:

wei = torch.ones((T, T)).tril() / torch.ones((T, T)).tril().sum(1, True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [ ]:
xbow2 = wei @ x
xbow2

tensor([[[ 1.4752, -0.3331],
         [-0.3116, -0.6225],
         [-0.0659,  0.3492],
         [ 0.3001,  0.4373],
         [ 0.3418,  0.3063],
         [ 0.3162,  0.5028],
         [ 0.5105,  0.2663],
         [ 0.4939,  0.1335]],

        [[-1.1295,  0.1419],
         [-1.5272, -0.7374],
         [-0.8819, -0.5697],
         [-0.5559, -0.3539],
         [-0.1231, -0.7248],
         [ 0.1501, -0.6569],
         [ 0.1923, -0.3422],
         [ 0.1837, -0.5312]],

        [[-0.1258, -1.4535],
         [-0.3433, -1.0400],
         [-0.4018,  0.1910],
         [-0.4511,  0.2944],
         [-0.2654,  0.4206],
         [-0.5179,  0.5138],
         [-0.5780,  0.4493],
         [-0.5998,  0.2087]],

        [[-1.0214, -0.2547],
         [-0.2451, -0.3431],
         [ 0.0966, -0.6000],
         [ 0.3138, -0.2802],
         [ 0.4017, -0.1623],
         [ 0.2028, -0.2646],
         [ 0.2550, -0.0770],
         [ 0.1895, -0.0277]]])

In [ ]:
torch.allclose(xbow1, xbow2)

True

### Method 3: (The one to use)/Sigmoid (Data-based weights):

In [ ]:
# Ab see upar sab theek hai ki we are taking the averages of all the previous tokens.
# But why assign equal weightages to all/ Its possible ki certain tokens mights find certain tokens more helpful/ more affinity/
# So, why not take a weighted average/ And the weights will update accordingly (in the training)

tril = torch.ones(T, T).tril()
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

You don't want the weights to be all zeros initially only. Something that you want to learn (data-dependent way). Some tokens will find some tokens more or less interesting.

Getting these weights is the problem that attention solves.

In [ ]:
wei = torch.zeros(T, T) # Something the model will learn.
wei

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
wei = wei.masked_fill(tril == 0, -float('inf')) # Future tokens can't talk. (Decoder)
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
wei.softmax(1) # Counts to probabilities/Conversion.
# Currently all previous tokens have the same weight. But the idea is that they should be learned during training.

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [ ]:
xbow3 = wei.softmax(1) @ x
xbow3

tensor([[[ 1.4752, -0.3331],
         [-0.3116, -0.6225],
         [-0.0659,  0.3492],
         [ 0.3001,  0.4373],
         [ 0.3418,  0.3063],
         [ 0.3162,  0.5028],
         [ 0.5105,  0.2663],
         [ 0.4939,  0.1335]],

        [[-1.1295,  0.1419],
         [-1.5272, -0.7374],
         [-0.8819, -0.5697],
         [-0.5559, -0.3539],
         [-0.1231, -0.7248],
         [ 0.1501, -0.6569],
         [ 0.1923, -0.3422],
         [ 0.1837, -0.5312]],

        [[-0.1258, -1.4535],
         [-0.3433, -1.0400],
         [-0.4018,  0.1910],
         [-0.4511,  0.2944],
         [-0.2654,  0.4206],
         [-0.5179,  0.5138],
         [-0.5780,  0.4493],
         [-0.5998,  0.2087]],

        [[-1.0214, -0.2547],
         [-0.2451, -0.3431],
         [ 0.0966, -0.6000],
         [ 0.3138, -0.2802],
         [ 0.4017, -0.1623],
         [ 0.2028, -0.2646],
         [ 0.2550, -0.0770],
         [ 0.1895, -0.0277]]])

In [ ]:
torch.allclose(xbow1, xbow3)

True

### How to get that weight matrix in a data-driven way?

In [ ]:
# Key & Query Vectors.
# Each token in time emmits a Key & Query vector. -> So, Key is like, what I have/who I am, and query is like what I am looking for.
# Idea is if we perform a dot product of Key & Query vectors/ Like ith position in time vaale token ke query vector ka picchle valo ke key vectors se.
# The more the value/ the more is affinity.

In [ ]:
# Current / Existing idea.

B, T, C = 4, 8, 32
tril = torch.ones((T, T)).tril()
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [ ]:
wei = torch.zeros(T, T) # This we want to learn in some way (Initial affinities).
wei

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

**(Decoder block)/ Future tokens can't communicate.
If we don't apply this then all tokens can fully communicate. Examples in Anomaly detection/ Sentiment analysis**

In [ ]:
wei = wei.masked_fill(tril == 0, float('-inf')) # Decoder block.
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
wei = wei.softmax(1) # Normalization/ (weights calculation) (T, T)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [ ]:
wei @ x # Currently the weights are also same for all the batches.

tensor([[[ 1.4752, -0.3331],
         [-0.3116, -0.6225],
         [-0.0659,  0.3492],
         [ 0.3001,  0.4373],
         [ 0.3418,  0.3063],
         [ 0.3162,  0.5028],
         [ 0.5105,  0.2663],
         [ 0.4939,  0.1335]],

        [[-1.1295,  0.1419],
         [-1.5272, -0.7374],
         [-0.8819, -0.5697],
         [-0.5559, -0.3539],
         [-0.1231, -0.7248],
         [ 0.1501, -0.6569],
         [ 0.1923, -0.3422],
         [ 0.1837, -0.5312]],

        [[-0.1258, -1.4535],
         [-0.3433, -1.0400],
         [-0.4018,  0.1910],
         [-0.4511,  0.2944],
         [-0.2654,  0.4206],
         [-0.5179,  0.5138],
         [-0.5780,  0.4493],
         [-0.5998,  0.2087]],

        [[-1.0214, -0.2547],
         [-0.2451, -0.3431],
         [ 0.0966, -0.6000],
         [ 0.3138, -0.2802],
         [ 0.4017, -0.1623],
         [ 0.2028, -0.2646],
         [ 0.2550, -0.0770],
         [ 0.1895, -0.0277]]])

In [ ]:
# Attention idea.

B, T, C = 4, 8, 32
x = torch.randn((B, T, C))

head_size = 16 # Hyper Parameter.
key = nn.Linear(C, head_size, bias = False)
query = nn.Linear(C, head_size, bias = False)
# A linear layer with input features as C (32) length of my token embeddings and the output features length is Head size
# C as input size because all tokens in Time T are represented by a vector of size C only right.

tril = torch.ones((T, T)).tril()
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [ ]:
# wei = torch.zeros((T, T)) # Initial affinities --> Ye nahi karna hai ab.
x.shape, key

(torch.Size([4, 8, 32]), Linear(in_features=32, out_features=16, bias=False))

In [ ]:
k = key(x) # Per batch har 8 tokens have a key vector.
k.shape, k

(torch.Size([4, 8, 16]),
 tensor([[[ 4.3157e-01,  7.4206e-01,  1.3768e-01,  7.3645e-02,  3.4282e-01,
            1.8170e-02,  6.4777e-01,  5.1194e-01,  3.3060e-01,  1.6556e-01,
           -1.1616e+00, -2.4909e-01, -7.1097e-01, -5.4473e-01, -9.3143e-01,
           -3.4398e-01],
          [-4.8183e-01,  1.1450e-01,  1.0956e+00, -5.6095e-01,  4.0720e-01,
           -1.0027e+00, -9.1336e-01, -2.1585e-01,  3.0891e-01, -3.1538e-02,
           -1.3119e+00, -9.0480e-01, -7.0076e-01,  6.8252e-01, -5.7555e-01,
           -3.5477e-01],
          [-8.6975e-01,  1.1035e+00,  4.0402e-01, -2.8101e-01, -5.0694e-01,
            2.0276e-01, -1.4554e-01,  4.7682e-01,  1.2649e+00,  6.2719e-01,
            2.2743e-01, -8.7544e-03, -6.1369e-01,  9.3223e-01,  1.1623e+00,
            6.7829e-01],
          [ 2.7745e-01, -8.5276e-01,  3.6115e-01,  7.8373e-01,  1.4838e-02,
           -5.0581e-01, -7.5652e-01,  5.8800e-02, -4.1435e-01,  1.4338e-03,
            2.8093e-03, -4.3071e-01,  5.6401e-01,  7.1772e-02,  

In [ ]:
q = query(x) # Per batch every token in Time T has a head-size (16) length query vector.
q.shape, q

(torch.Size([4, 8, 16]),
 tensor([[[ 8.2798e-02,  3.7758e-01, -1.5011e-01, -1.6374e-01,  8.0848e-01,
            4.0795e-01,  2.6162e-01,  6.9286e-02,  3.4164e-01,  7.5425e-02,
           -3.6533e-01,  3.0754e-01, -2.5684e-01,  3.8492e-01, -4.5687e-01,
            4.6824e-01],
          [-7.7159e-02,  1.1028e+00,  2.3982e-01, -7.7130e-02,  5.9606e-01,
           -4.7385e-01,  1.3238e+00, -3.9902e-01, -8.4110e-01,  1.1358e+00,
            1.9339e-01,  1.2161e-01,  1.1745e+00, -3.7020e-01,  6.9384e-01,
            1.1583e-01],
          [-1.5508e+00, -1.0002e+00,  4.0072e-01, -1.6033e+00, -1.1085e-01,
           -1.1220e+00, -3.3912e-02, -3.9096e-01, -7.7762e-02,  3.7208e-01,
           -1.2672e+00,  1.0435e+00,  2.7223e-01,  3.0767e-01, -5.5513e-01,
            9.9991e-01],
          [-2.0473e-01,  3.6583e-01,  8.3551e-01,  1.1878e-01, -2.2045e-01,
           -1.1245e-01, -1.7858e-01,  3.0638e-01,  3.3029e-01,  2.6949e-01,
           -2.1673e-01, -1.1344e-01,  3.5774e-01, -2.2349e-01,  

In [ ]:
# Ab inhe multiple karna hai. Like q at i for all the keys before i.
# And result mujhe T, T me laana hai (8, 8)

k.shape, q.shape

(torch.Size([4, 8, 16]), torch.Size([4, 8, 16]))

In [ ]:
# Simplify-> 1 batch.

# q1 q1
# q2 q2

# k1 k1 k1 k1 -> Key vector for the first token.
# k2 k2 k2 k2 -> Key vector for the second token.

q[0].shape, k[0].shape

(torch.Size([8, 16]), torch.Size([8, 16]))

In [ ]:
k[0].T.shape # Transpose keys.
# k1 k2
# k1 k2
# k1 k2
# k1 k2

torch.Size([16, 8])

Every query of my ith is being multiplied by the key of others in the past.

In [ ]:
# Exactly what you want.
(q[0] @ k[0].T) # -> Every query in a row is being multipled with evry key of its own and previous.

tensor([[ 1.4833e+00,  4.4066e-01,  6.9455e-01, -1.5006e+00, -1.1953e+00,
         -5.6557e-01,  5.2630e-01,  4.1084e-01],
        [-3.1310e-03, -2.1112e+00,  1.3183e-01, -5.1706e-01,  2.6835e-01,
          7.7266e-01,  1.1052e+00, -3.9765e-01],
        [-6.9502e-01,  3.8330e+00,  4.9514e-01, -7.0815e-01, -2.5026e+00,
          3.3687e-01,  8.1964e-01,  1.6262e+00],
        [ 5.6323e-01,  1.1812e+00,  1.2683e+00,  3.3366e-01,  6.9156e-01,
          1.2798e+00, -3.9815e-01,  7.4339e-01],
        [-5.3599e-01, -1.5408e+00, -6.0025e-01,  7.7762e-01,  1.1905e+00,
          5.5721e-01, -9.1375e-01, -3.9100e-01],
        [-2.1092e-01,  2.6348e+00, -1.6495e+00,  9.9660e-01,  9.4299e-01,
         -3.7353e-01, -1.1364e-01,  1.1746e+00],
        [-1.4393e+00, -1.8231e+00,  1.1462e+00,  2.0692e-01, -2.1074e-02,
          9.2613e-01, -1.6808e-01, -1.4690e+00],
        [-3.9401e-01,  1.8549e+00,  2.4465e+00, -1.1454e-01,  7.9159e-02,
          1.9028e+00,  5.6284e-01,  2.4558e-01]], grad_fn=<MmBack

In [ ]:
# Now we want to broadcast this operation across batches.

k.shape, k.transpose(-3, -2).shape, k.transpose(-2, -1).shape, k.transpose(-1, -3).shape

(torch.Size([4, 8, 16]),
 torch.Size([8, 4, 16]),
 torch.Size([4, 16, 8]),
 torch.Size([16, 8, 4]))

In [ ]:
(q @ k.transpose(-2, -1)).shape # Now we will have different T, T weights for all the batches.
# Which is even better. As every batch is a bit different, right.

torch.Size([4, 8, 8])

In [ ]:
# Operation across batches.

wei = q @ k.transpose(-2, -1)
wei

tensor([[[ 1.4833e+00,  4.4066e-01,  6.9455e-01, -1.5006e+00, -1.1953e+00,
          -5.6557e-01,  5.2630e-01,  4.1084e-01],
         [-3.1310e-03, -2.1112e+00,  1.3183e-01, -5.1706e-01,  2.6835e-01,
           7.7266e-01,  1.1052e+00, -3.9765e-01],
         [-6.9502e-01,  3.8330e+00,  4.9514e-01, -7.0815e-01, -2.5026e+00,
           3.3687e-01,  8.1964e-01,  1.6262e+00],
         [ 5.6323e-01,  1.1812e+00,  1.2683e+00,  3.3366e-01,  6.9156e-01,
           1.2798e+00, -3.9815e-01,  7.4339e-01],
         [-5.3599e-01, -1.5408e+00, -6.0025e-01,  7.7762e-01,  1.1905e+00,
           5.5721e-01, -9.1375e-01, -3.9100e-01],
         [-2.1092e-01,  2.6348e+00, -1.6495e+00,  9.9660e-01,  9.4299e-01,
          -3.7353e-01, -1.1364e-01,  1.1746e+00],
         [-1.4393e+00, -1.8231e+00,  1.1462e+00,  2.0692e-01, -2.1074e-02,
           9.2613e-01, -1.6808e-01, -1.4690e+00],
         [-3.9401e-01,  1.8549e+00,  2.4465e+00, -1.1454e-01,  7.9159e-02,
           1.9028e+00,  5.6284e-01,  2.4558e-01]],

In [ ]:
# Bradcasting across batches.

wei = wei.masked_fill(tril == 0, float('-inf')) # Decoder mechanism. (Future tokens can't communicate)/ Which makes sense here.
wei

tensor([[[ 1.4833e+00,        -inf,        -inf,        -inf,        -inf,
                 -inf,        -inf,        -inf],
         [-3.1310e-03, -2.1112e+00,        -inf,        -inf,        -inf,
                 -inf,        -inf,        -inf],
         [-6.9502e-01,  3.8330e+00,  4.9514e-01,        -inf,        -inf,
                 -inf,        -inf,        -inf],
         [ 5.6323e-01,  1.1812e+00,  1.2683e+00,  3.3366e-01,        -inf,
                 -inf,        -inf,        -inf],
         [-5.3599e-01, -1.5408e+00, -6.0025e-01,  7.7762e-01,  1.1905e+00,
                 -inf,        -inf,        -inf],
         [-2.1092e-01,  2.6348e+00, -1.6495e+00,  9.9660e-01,  9.4299e-01,
          -3.7353e-01,        -inf,        -inf],
         [-1.4393e+00, -1.8231e+00,  1.1462e+00,  2.0692e-01, -2.1074e-02,
           9.2613e-01, -1.6808e-01,        -inf],
         [-3.9401e-01,  1.8549e+00,  2.4465e+00, -1.1454e-01,  7.9159e-02,
           1.9028e+00,  5.6284e-01,  2.4558e-01]],

### Dim = 1 or -1 for softmax? Answer: -1.

In [ ]:
wei[0].softmax(dim = 1), wei[0].softmax(dim = 0)

(tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.8917, 0.1083, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0103, 0.9557, 0.0339, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1762, 0.3270, 0.3567, 0.1401, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0859, 0.0314, 0.0805, 0.3194, 0.4827, 0.0000, 0.0000, 0.0000],
         [0.0387, 0.6668, 0.0092, 0.1296, 0.1228, 0.0329, 0.0000, 0.0000],
         [0.0260, 0.0177, 0.3448, 0.1348, 0.1073, 0.2767, 0.0926, 0.0000],
         [0.0222, 0.2107, 0.3808, 0.0294, 0.0357, 0.2211, 0.0579, 0.0422]],
        grad_fn=<SoftmaxBackward0>),
 tensor([[0.4422, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1000, 0.0017, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0501, 0.6573, 0.0795, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1762, 0.0464, 0.1723, 0.1661, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0587, 0.0030, 0.0266, 0.2590, 0.4153, 0.0000, 0.00

In [ ]:
# Across batches.

wei = wei.softmax(dim = -1)
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.8917, 0.1083, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0103, 0.9557, 0.0339, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1762, 0.3270, 0.3567, 0.1401, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0859, 0.0314, 0.0805, 0.3194, 0.4827, 0.0000, 0.0000, 0.0000],
         [0.0387, 0.6668, 0.0092, 0.1296, 0.1228, 0.0329, 0.0000, 0.0000],
         [0.0260, 0.0177, 0.3448, 0.1348, 0.1073, 0.2767, 0.0926, 0.0000],
         [0.0222, 0.2107, 0.3808, 0.0294, 0.0357, 0.2211, 0.0579, 0.0422]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.9368, 0.0632, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0297, 0.9610, 0.0093, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0030, 0.1021, 0.0341, 0.8607, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1088, 0.2621, 0.2497, 0.1813, 0.1981, 0.0000, 0.0000, 0.0000],
         [0.2146, 0.013

In [ ]:
# Across batches.

wei = wei.softmax(1)
wei

tensor([[[2.0071e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [2.5848e-01, 8.8304e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [6.5133e-02, 7.6257e-02, 3.2374e-01, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [3.4626e-02, 8.6648e-02, 8.7932e-02, 1.7886e-01, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [2.6026e-01, 1.2114e-01, 1.9711e-01, 8.5972e-02, 2.8101e-01,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [5.7162e-02, 1.1309e-01, 1.0710e-01, 3.5905e-01, 2.0909e-01,
          2.4342e-01, 0.0000e+00, 0.0000e+00],
         [7.8271e-02, 9.8534e-02, 2.1223e-01, 2.3451e-01, 9.3926e-02,
          2.3527e-01, 6.6809e-01, 0.0000e+00],
         [4.5354e-02, 4.1603e-01, 7.1894e-02, 1.4162e-01, 4.1598e-01,
          5.2131e-01, 3.3191e-01, 1.0000e+00]],

        [[2.0877e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.00

In [ ]:
# 1 way is

wei.shape, x.shape

(torch.Size([4, 8, 8]), torch.Size([4, 8, 32]))

In [ ]:
wei @ x

tensor([[[-1.8460e-01, -1.4461e-01,  9.0019e-02,  ...,  1.7141e-01,
          -4.9286e-02,  6.7666e-02],
         [-3.1187e-01, -1.9069e-01,  8.5496e-03,  ...,  5.0071e-02,
          -4.4716e-02,  9.9576e-02],
         [-1.1831e-01, -2.7097e-01, -2.2114e-01,  ..., -5.0821e-01,
           5.3429e-01,  1.5294e-01],
         ...,
         [-2.9712e-01, -2.8764e-01,  2.6062e-01,  ...,  1.0546e-01,
           4.2910e-01, -1.3861e-01],
         [-2.2432e-02, -9.1413e-01, -7.7480e-01,  ...,  7.5201e-01,
           9.6961e-01, -2.6405e-02],
         [-2.1500e-01, -2.0906e+00, -1.9840e-01,  ...,  1.3441e+00,
          -3.5827e-01,  5.3981e-01]],

        [[-3.0760e-01, -3.7542e-01, -1.0754e-01,  ..., -5.9489e-02,
          -1.4631e-02,  2.1001e-02],
         [-1.1553e-01, -1.3798e-01, -4.7470e-02,  ..., -3.6192e-02,
           9.5579e-03,  8.3032e-03],
         [-6.5852e-02,  1.4588e-01,  1.6687e-02,  ..., -2.8465e-01,
           4.3099e-01, -5.2758e-01],
         ...,
         [-4.3223e-01, -5

**One more important thing for a single self attention head**

The product of query & key vectors -> Weights
We don't * Weights directly with X
We bring in other weights/parameters -> Values. (This is what I have, I am looking for and this is what I will contribute)/ Think of X has hidden/private

In [ ]:
# Value is what gets aggregated in terms of this single head between the different nodes/tokens. What I will communicate.

value = nn.Linear(C, head_size, bias = False)
value

Linear(in_features=32, out_features=16, bias=True)

In [ ]:
v = value(x)
v.shape

torch.Size([4, 8, 16])

In [ ]:
wei.shape

torch.Size([4, 8, 8])

In [ ]:
output = wei @ v
output.shape

torch.Size([4, 8, 16])

### Attention Theory:

1. Attention is a communication mechanism (Nodes in a directed graph)/connected with each other. These nodes have some information vectors and you are just doing sorts of weighted average (in a data way/for the incoming nodes).

2. There is no notion of space here. Aap ko khud se positional embeddings of tokens daalni hogi.

3. Batch-wise saare operations hote hai/ Vo independent of each hai. At max only 8 tokens are communicating.

4. Attention works on any graph/ And our Language model ke case me it is like
1->self1->2->1->self2->3->1->2->self3 like this.

5. Self Vs Cross Attention: Self attention me all Key, Queries, and Values, are all obtained from x. While in Cross attention. Query(x) comes from x, while we bring in keys and values from some other sources/encoding blocks.
(Self-serving nodes.)

6. Attention supports both/ Encoding and Decoding mechanism. (Like if the Future tokens can talk with the past ones or not.)

```wei.masked_fill(tril == 0, float('inf'))```
Ye line hata do to all tokens can fully communicate (Anomaly detction/Sentiment analysis with TFs)

7. Scaled Self-Attention comes from the Research Paper (Attention is all you need).
Output/Attention(Key, Queries, Values) = Sigmoid(Queries * Keys.T) * Values / (Head_Size)**0.5



### Why Scaled Attention?

1. Weights (Queries * Keys.T) ke variance ko unit karne ke liye.
2. Yadi weights high ho/ To unpar softmax lagane se one-hot vectors ban jaaege/ High probabilities, to only limited tokens you are considering meaningful, which you don't want atleast initially.

In [ ]:
B, T, C = 4, 8, 32
head_size = 16

k = torch.randn((B, T, head_size))
q = torch.randn((B, T, head_size))
k.shape, q.shape

(torch.Size([4, 8, 16]), torch.Size([4, 8, 16]))

In [ ]:
wei = q @ k.transpose(-2, -1)
wei.shape, wei

(torch.Size([4, 8, 8]),
 tensor([[[ 4.7481e+00, -2.4683e+00, -1.0876e+00, -9.7772e+00, -2.3396e-01,
            6.1394e+00,  3.3382e+00,  2.2931e+00],
          [ 2.7978e-02, -3.1605e+00,  1.2273e+00, -5.1468e+00,  3.1734e+00,
            3.2849e+00,  1.0947e+01, -2.7826e+00],
          [ 2.9049e+00,  1.9506e+00, -2.7506e+00,  1.6472e+00, -6.7642e-01,
           -2.0537e-01, -3.6162e-01, -8.8007e-01],
          [ 8.0480e-01, -4.1009e+00,  8.8652e-01, -4.5195e+00,  5.3437e+00,
           -1.0637e+00,  3.1171e+00,  2.6227e+00],
          [-5.2459e+00,  2.4201e+00, -3.4533e+00, -2.0364e+00,  2.0968e+00,
           -2.2742e+00,  4.3225e+00,  2.8033e+00],
          [ 2.9932e+00, -2.6038e+00,  2.6847e+00, -7.3418e-01, -3.2006e+00,
           -1.9112e+00,  2.2291e+00,  1.9676e+00],
          [-5.7295e+00,  3.0572e+00,  5.9835e+00,  6.3362e+00, -6.7752e+00,
            7.3328e+00, -2.6064e-01, -2.0147e+00],
          [ 4.1902e-01, -1.9252e+00, -1.2346e+00,  1.3810e+00,  3.1504e-01,
           

In [ ]:
k.var(), q.var(), wei.var() # So, the variance of weights is of the order of head_size
# Is se bachne ke liye we divide it by sqrt(head_size)

(tensor(1.0057), tensor(0.9344), tensor(15.4895))

In [ ]:
(wei / (head_size)**0.5).var()

tensor(0.9681)

In [ ]:
wei

tensor([[[ 4.7481e+00, -2.4683e+00, -1.0876e+00, -9.7772e+00, -2.3396e-01,
           6.1394e+00,  3.3382e+00,  2.2931e+00],
         [ 2.7978e-02, -3.1605e+00,  1.2273e+00, -5.1468e+00,  3.1734e+00,
           3.2849e+00,  1.0947e+01, -2.7826e+00],
         [ 2.9049e+00,  1.9506e+00, -2.7506e+00,  1.6472e+00, -6.7642e-01,
          -2.0537e-01, -3.6162e-01, -8.8007e-01],
         [ 8.0480e-01, -4.1009e+00,  8.8652e-01, -4.5195e+00,  5.3437e+00,
          -1.0637e+00,  3.1171e+00,  2.6227e+00],
         [-5.2459e+00,  2.4201e+00, -3.4533e+00, -2.0364e+00,  2.0968e+00,
          -2.2742e+00,  4.3225e+00,  2.8033e+00],
         [ 2.9932e+00, -2.6038e+00,  2.6847e+00, -7.3418e-01, -3.2006e+00,
          -1.9112e+00,  2.2291e+00,  1.9676e+00],
         [-5.7295e+00,  3.0572e+00,  5.9835e+00,  6.3362e+00, -6.7752e+00,
           7.3328e+00, -2.6064e-01, -2.0147e+00],
         [ 4.1902e-01, -1.9252e+00, -1.2346e+00,  1.3810e+00,  3.1504e-01,
          -5.0383e+00, -7.6571e-01,  2.5307e+00]],

In [ ]:
# So, apply scaling/ division by the sqrt(head_size) first and then apply softmax.

# Master Code (Similar to the Research Paper: Attention is all you need):

## Base/Platform:

1. Self attention can't tolerate very high learning rates.

2. Even reduce (3e-4) because of a very highly complex model.

In [ ]:
# Module list holds a list of layers/ nn.Module objects.
# Pytorch handles (parameters) and updation at the backend.

nn.ModuleList([nn.Linear(1, 1), nn.Linear(1, 1)])

ModuleList(
  (0-1): 2 x Linear(in_features=1, out_features=1, bias=True)
)

In [ ]:
nn.Sequential(*[nn.Linear(3, 3)]*2)

Sequential(
  (0): Linear(in_features=3, out_features=3, bias=True)
  (1): Linear(in_features=3, out_features=3, bias=True)
)

## Optimization and other pointers (Mentioned in the Research Paper):

1. Residual/Skip connections (which solves vanishing gradient problem). During B.P as straight highway for the loss to propagate.
x = x (original/highway) + x (passed through the network)

Allow deeper networks to learn effectively.
(Gradient atakta nahi hai in the deep layers)

2. Certain Projection layers.

**I guess idea is after multi head attention and Feed forward you have a few Linear layers to change the dimension/(projection). Possibly like agar head_size and n_embed alag-lag hai.**

In MultiHeadAttention, after concatenating the heads, you typically apply a linear transformation to project the concatenated result back into the original embedding space. This linear layer isn't a skip connection by itself but is used to reduce or change the dimensionality after combining the attention heads. It can be seen as a "projection" because it adjusts the output space after attention.

In FeedForward, applying a linear layer also isn't a skip connection. It's just a transformation applied after the activation function (like ReLU), often used to map the output to a desired size or shape.

**Feed Forward network me according to paper x -> 4x and back from 4x to x kind of projection hoti hai.**

---

Avoid overfitting.


3. Layer Normalization.

Idea is jo actiavtions/data/matrices flowing through the network, sometimes, they may explode/blow up or vanish (jis se training sahi se nahi ho paati). Slow down/unstable.

Normalize activations across a feature (training faster and generalize better). In case of Language models -> Ye hum -> across each token (Channel dimension pe karte hai.)

**Pre-norm formulation: Slight deviation from the Research Paper.**
The layer norms are applied before the Multi-Head Attention Block and the Computation one (Feed forward).

**One layer norm after all the Transformer blocks -> before feeding it to the LM Head (Jaha pe projection hogi towards -> Vocabulary and Softmax calculations)**


Layer norms have some trainable parameters.

4. Dropout (Generic).

* Dropout after projection layer in Multi-Head Attention
* Dropout after projection layer in FFWD N/W
* Dropout after the weights matrix calculation in Single Self-Attention Head (B, T, T)

## Code (Evolution):

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
torch.cuda.is_available() # GPU check.

False

In [ ]:
# Hyperparameters.

batch_size = 64 # # Samples/Sequences of Input tokens to process in ||. (NN batch inputs).
block_size = 256 # Context Window (Time dimension).
epochs = 5000 # One complete pass of the training data set.
eval_interval = 500 # After how many iterations/epochs we will compute the loss. (Andrej K's logic).
eval_iters = 200 # How many unique samples of the training and validation data to use to compute the loss (Since every sample is more/less noisy).
learning_rate = 3e-4 # Self-attention can't tolerate very high learning rates. (Even reduce/ Highly complex model)
device = 'cuda' if torch.cuda.is_available() else 'cpu' # GPU VS CPU.
n_embed = 384 # Embedding vector length.
n_heads = 6 # Number of Single self-attention heads in the Mutli-head attention block.
n_layers = 6 # Number of Transformer blocks (Multi-head attention + Feed forward) # (Communication + Build on it (Computation)).
dropout = 0.2 # Dropout rate.

# Reproducibility.
torch.manual_seed(1337)

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-03-05 06:25:26--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-03-05 06:25:27 (18.2 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open('input.txt', 'r') as file:
  text = file.read()

print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [ ]:
vocabulary = sorted(list(set(text))) # All the possible tokens, the model can see and emmit.
vocab_size = len(vocabulary) # Number of distinct tokens.

stoi = {s:i for i, s in enumerate(vocabulary)}
itos = {i:s for i, s in enumerate(vocabulary)}

encode = lambda text: [stoi[s] for s in text]
decode = lambda encoding: ''.join([itos[i] for i in encoding])

In [ ]:
encode('Hi!'), decode([20, 47, 2])

([20, 47, 2], 'Hi!')

In [ ]:
data = torch.tensor(encode(text))
data

tensor([18, 47, 56,  ..., 45,  8,  0])

In [ ]:
len(data), len(text)

(1115394, 1115394)

In [ ]:
# Training and Validation data split.

split_size = int(0.9 * len(data))
training_data = data[:split_size]
validation_data = data[split_size:]
len(training_data) + len(validation_data)

1115394

In [ ]:
# Get the batches of data (X and Y).
# 1. Training data (X and Y) ko we will move to the device.

def get_batch(data_type):
  _data = training_data if data_type == 'train' else validation_data
  idxs = torch.randint(0, len(_data) - block_size, (batch_size, )) # Batch size number of random indeces.
  xb = torch.stack([_data[idx: idx + block_size] for idx in idxs], dim = 0)
  yb = torch.stack([_data[idx + 1: idx + block_size + 1] for idx in idxs], dim = 0)

  # Move the training data to device (GPU/CPU).
  xb = xb.to(device) # Re-initialization is required.
  yb = yb.to(device)

  return xb, yb

**GO THROUGH FROM BOTTOM TO TOP:**

NOTE:

```n_embed = head_size```

Overall head_size after concatenation inside the Multi-Head Attention Block.

In [ ]:
# Intersperse the communication (Bikher do/open multiple communication channels with Multi-Head Attention & then computation (FFWDN))
class Block(nn.Module):
  '''Transformer Block (Communication followed by Computation).'''
  def __init__(self, n_heads, head_size):
    super().__init__()
    # head_size is the final head size that is required after concatenation (which will be equal to n_embed only).
    '''Layer Norm (Across Channel/For all tokens)'''
    self.ln1 = nn.LayerNorm(n_embed)
    self.mha = MultiHeadAttention(n_heads, head_size)
    self.ln2 = nn.LayerNorm(n_embed)
    self.ffwd = FeedForward()

  def forward(self, x):
    # x -> B, T, C (n_embed) -> B, T, C (head_size or n_embed) (After MHA) -> B, T, C (head_size or n_embed) -> After Feed Forward.
    '''
    Residual/Skip Connection. (x + )
    Pre-Norm Formulation (Layer norm before Multi-head attention and FFWD)
    '''
    x = x + self.mha(self.ln1(x))
    return x + self.ffwd(self.ln2(x))




# You opened a lot of communication channels to allow the tokens to talk (Multi-Head Attention).
# But you need to learn/work on that knowledge and then move to logits.
# Basically lm_head (final projection to the vocabulary) ke pehle complexity add kar rahe hai.
class FeedForward(nn.Module):
  '''A simple feed forward NN after the Multi-Head Attention.'''
  def __init__(self):
    super().__init__()
    # 1 X 4 X 1 is suggested in the Research Paper.
    self.nn = nn.Sequential(
        nn.Linear(n_embed, 4 * n_embed), # An extra layer before the language model head.
        nn.ReLU(),
        nn.Linear(4 * n_embed, n_embed),  # Projection layer (added here only).
        nn.Dropout(dropout) # Dropout layer.
    )

  def forward(self, x):
    # x -> (B, T, C (n_embed)) -> (B, T, C (n_embed))
    return self.nn(x)




# Multiple Self-Attention Heads in parallel and their concatenation.
class MultiHeadAttention(nn.Module):
  '''Opening multiple channels of communication (via multiple single self-attention heads), since these tokens have a lot to talk about.'''
  def __init__(self, n_heads, head_size):
    super().__init__()
    # head_size is the final head size that is required after concatenation (which will be equal to n_embed only).
    # Fir usi par next block operate karega.
    self.sa_heads = nn.ModuleList([Head(head_size // n_heads) for _ in range(n_heads)]) # A list of Self-attention heads.

    self.projection = nn.Linear(head_size, n_embed) # Dono same hi hai. Jaan ke alag-lag likha hai.
    # A projection layer to change the dimensionality.

    '''Dropout layer after the projections.'''
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    # x -> B, T, C (n_embed) -> (B, T, C (head_size)/(n_heads))*n_heads
    '''
    return torch.concat([sa_head(x) for sa_head in self.sa_heads], dim = -1) # Along the last axis (C vaali). (B, T, C (head_size)/(n_heads))*n_heads
    '''
    out = torch.concat([sa_head(x) for sa_head in self.sa_heads], dim = -1) # Along the last axis (C vaali). (B, T, C (head_size)/(n_heads))*n_heads
    return self.dropout(self.projection(out)) # (B, T, C (head_szie)) -> (B, T, C (n_embed)) # Both are same. Neeche Language Model me jo Blocks ke constructor me call hua hai.
    # ++ self.dropout() at the end.




# A Single Self-Attention Head/Block.
class Head(nn.Module):
  '''Single Self-Attention Head.'''

  def __init__(self, head_size):
    super().__init__()
    # Key, Query, and Value vectors per token.
    self.key = nn.Linear(n_embed, head_size, bias = False) # They will operate on x, right, and x is B, T, C (n_embed)
    self.query = nn.Linear(n_embed, head_size, bias = False)
    self.value = nn.Linear(n_embed, head_size, bias = False)

    # Fixed size ka lower triangular matrix hai, but we will slice it on the basis of T (The current tokens in time in x).
    self.register_buffer('tril', torch.ones((block_size, block_size)).tril()) # Not to be considered as a model parameter. It is a buffer.

    self.head_size = head_size # Trying to pass constant values to the forward/any other method.

    '''Dropout layer after the previous/other tokens' weights learned.'''
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B, T, C = x.shape # (B, T, C (n_embed)) # T can vary between 1 to block_size.

    k = self.key(x) # (B, T, C (n_embed)) @ (n_embed, head_size) -> (B, T, C (head_size))
    q = self.query(x)
    v = self.value(x)

    wei = q @ k.transpose(-2, -1) * (self.head_size**(-0.5)) # Scaled self-attention # (B, T, C (head_size)) @ (B, C(head_size), T) -> (B, T, T)
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # Decoder mechanism (Future tokens can't talk). (B, T, T)
    wei = wei.softmax(dim = -1) # Along the last dimension. (B, T, T)

    wei = self.dropout(wei) # Dropout layer.

    return wei @ v # (B, T, T) @ (B, T, C (head_size)) -> (B, T, C (head_size))




# A Large Language Model to replicate the Research Paper (Attention is all you need!) structure.
class LanguageModel(nn.Module):
  '''Large Language Model.'''

  def __init__(self):
    super().__init__()
    # Attributes.
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed) # [Number of embeddings: vocab_size (65), C (Embedding dimension) = n_embed]
    self.position_embedding_table = nn.Embedding(block_size, n_embed) # [Number of embeddings: block_size, C (n_embed)]. An embedding per position.

    '''
    # Single Self-Attention Head.
    self.sa_head = Head(head_size)

    # Multi-Head Attention Block.
    self.sa_heads = MultiHeadAttention(n_heads, head_size)

    # Feed-Forward Network.
    self.ffwd = FeedForward() # One Linear layer -> B, T, C (head_size) -> B, T, C (head_size) + Relu -> B, T, C (vocab_size)
    '''

    # Transformer Blocks: Multi-Head Attention + Computation.
    # This syntax will get the individual entries/layers/models from the list and pass it to the nn.Sequential model.
    self.blocks = nn.Sequential(*[Block(n_heads, n_embed) for _ in range(n_layers)]) # Passing in the final head_size required.

    '''Layer Norm before the final projections to the vocabulary.'''
    self.ln = nn.LayerNorm(n_embed)

    # Language Model Head (To project back to the vocab_size for output).
    self.lm_head = nn.Linear(n_embed, vocab_size) # Get the vocab size features (all possible tokens) back from the embedding dimension.
    # Ek alag se LM head chaahiye to convert head_size (n_embed) -> vocab_size.

  def forward(self, x, y = None):
    B, T = x.shape # T can be anything between 1 to block_size.

    token_embeddings = self.token_embedding_table(x) # (B, T) -> (B, T, C (n_embed))
    # position_embeddings = self.position_embedding_table(torch.arange(block_size)) -> No, not the block size because input tokens can be less than block_size,
    # right.
    position_embeddings = self.position_embedding_table(torch.arange(T, device = device)) # (T, ) -> (T, C (n_embed))
    # Perhaps we are creating a new tensor here (torch.arange(T)) and need to move that to device as well.
    x = token_embeddings + position_embeddings # (B, T, C (n_embed)) + (T, C (n_embed))

    '''
    # Single Self-Attention Head.
    x = self.sa_head(x) # (B, T, C (n_embed)) -> (B, T, C (head_size))

    # Multi-Head Attention Block.
    x = self.sa_heads(x) # (B, T, C (n_embed)) -> (B, T, C (n_heads * head_size/n_heads)) -> (B, T, C (head_size))

    # Feed-Forward Network.
    logits = self.ffwd(x) # (B, T, C (head_size)) -> (B, T, C (vocab_size))
    '''

    # Transformer Blocks.
    x = self.blocks(x) # (B, T, C (n_embed)) -> (B, T, C (head_size)) # And dono same hai.

    # Final Layer Norm.
    x = self.ln(x) # (B, T, C (n_embed or head_size)) But standardized across the C (-1) dimension.

    # Language Model head.
    logits = self.lm_head(x) # (B, T, C (n_embed)) -> (B, T, C (vocab_size = 65))

    if y is None:
      loss = None
    else:
      loss = F.cross_entropy(logits.view(B * T, vocab_size), y.view(B * T))

    return logits, loss

  def generate(self, x, max_tokens):
    '''
    # Bi-Gram model.
    for _ in range(max_tokens):
      logits, loss = self(x) # (B, T, C (vocab_size))
      logits = logits[:, -1, :] # (B, C (vocab_size)) -> Last token in the Time dimension.

      probabilities = logits.softmax(dim = 1) # (B, C)
      x_next = torch.multinomial(probabilities, 1, replacement = True) # (B, 1)
      x = torch.concat([x, x_next], dim = 1) # (B, T) + (B, 1) -> (B, T + 1)
    '''

    for _ in range(max_tokens):
      x_block = x[:, -block_size:] # Restricting to only the last block_size tokens. (B, T)
      logits, loss = self(x_block) # (B, T, C (vocab_size))
      logits = logits[:, -1, :] # (B, C (vocab_size)) -> Last token in the Time dimension.

      probabilities = logits.softmax(dim = 1) # (B, C)
      x_next = torch.multinomial(probabilities, 1, replacement = True) # (B, 1)
      x = torch.concat([x, x_next], dim = 1) # (B, T) + (B, 1) -> (B, T + 1)

    return x

In [ ]:
# Validating that self.head_size:
# To Buffer and Constant/No-Tensors work.

head = Head(16)
head

Head(
  (key): Linear(in_features=384, out_features=16, bias=False)
  (query): Linear(in_features=384, out_features=16, bias=False)
  (value): Linear(in_features=384, out_features=16, bias=False)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [ ]:
for parameter in head.parameters():
  print(parameter.shape) # Only key, query, and value vectors (Only weights. No Biases).

torch.Size([16, 384])
torch.Size([16, 384])
torch.Size([16, 384])


In [ ]:
@torch.no_grad() # Ensure/Explicitly telling that do not involve/consider any calculations here for the gradient computation.
def estimate_loss():
  model.eval() # Model is in the evaluation phase.

  losses = {
      'train': torch.zeros((eval_iters, )),
      'val': torch.zeros((eval_iters, ))
  }
  for data_type in losses:
    for i in range(eval_iters):
      xb, yb = get_batch(data_type)
      logits, loss = model(xb, yb)
      losses[data_type][i] = loss.item()
    losses[data_type] = losses[data_type].mean().item()

  model.train() # Model is back in the training phase.

  return losses

In [ ]:
# 2. Model ko we will move to device.

model = LanguageModel()
model = model.to(device)
model

LanguageModel(
  (token_embedding_table): Embedding(65, 384)
  (position_embedding_table): Embedding(256, 384)
  (blocks): Sequential(
    (0): Block(
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (mha): MultiHeadAttention(
        (sa_heads): ModuleList(
          (0-5): 6 x Head(
            (key): Linear(in_features=384, out_features=64, bias=False)
            (query): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (projection): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ln2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (ffwd): FeedForward(
        (nn): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features

In [ ]:
torch.randn((3, 2)).numel()

6

In [ ]:
help(torch.numel) # Number of elements.

Help on built-in function numel in module torch:

numel(...)
    numel(input) -> int
    
    Returns the total number of elements in the :attr:`input` tensor.
    
    Args:
        input (Tensor): the input tensor.
    
    Example::
    
        >>> a = torch.randn(1, 2, 3, 4, 5)
        >>> torch.numel(a)
        120
        >>> a = torch.zeros(4,4)
        >>> torch.numel(a)
        16



In [ ]:
# Very important to visualize as to what is happening.

for parameters in model.parameters():
  print(parameters.shape)

torch.Size([65, 384])
torch.Size([256, 384])
torch.Size([384])
torch.Size([384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([384, 384])
torch.Size([384])
torch.Size([384])
torch.Size([384])
torch.Size([1536, 384])
torch.Size([1536])
torch.Size([384, 1536])
torch.Size([384])
torch.Size([384])
torch.Size([384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 384])
torch.Size([64, 3

In [ ]:
# Our model has roughly 11 Million parameters. WOW!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

sum([parameter.numel() for parameter in model.parameters()]) / 1e6

10.788929

In [ ]:
# Optimizer.

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0003
    maximize: False
    weight_decay: 0.01
)

In [ ]:
# Training loop.

# epochs = 2000
# eval_interval = 10
# eval_iters = 2
for i in range(epochs):
  xb, yb = get_batch('train') # Get a batch of my training data.
  logits, loss = model(xb, yb) # Forward pass.

  optimizer.zero_grad(set_to_none = True) # Set gradients to zero.
  loss.backward() # Back propagation of loss/ Compute gradients.
  optimizer.step() # Update all parameters.

  if (i % eval_interval) == 0 or (i == epochs - 1):
    losses = estimate_loss()
    print(f"Epoch: {i} Training Loss: {losses['train']:.4f} Validation Loss: {losses['val']:.4f}")

Epoch: 0 Training Loss: 3.5282 Validation Loss: 3.5493
Epoch: 500 Training Loss: 1.8810 Validation Loss: 1.9957
Epoch: 1000 Training Loss: 1.5328 Validation Loss: 1.7204
Epoch: 1500 Training Loss: 1.3953 Validation Loss: 1.6077
Epoch: 2000 Training Loss: 1.3086 Validation Loss: 1.5537
Epoch: 2500 Training Loss: 1.2519 Validation Loss: 1.5178
Epoch: 3000 Training Loss: 1.2020 Validation Loss: 1.4974
Epoch: 3500 Training Loss: 1.1597 Validation Loss: 1.4808
Epoch: 4000 Training Loss: 1.1222 Validation Loss: 1.4816
Epoch: 4500 Training Loss: 1.0850 Validation Loss: 1.4761
Epoch: 4999 Training Loss: 1.0497 Validation Loss: 1.4937


In [ ]:
# Inference.
# 3. Move the testing data to the device

context = torch.zeros((1, 1), dtype = torch.int, device = device) # Move the testing data onto device. Starting with a new line character.
print(decode(model.generate(context, 500)[0].tolist()))


But with price of tribe steels unwhanty
merfly.

Officion:
Treason, and first are more talions,
Provost in the hour, no more honour'd moan;
But sound my slavise to begin your hands.

Clown:
I have been a clog botten of right;
Men the dischanged your own way, here is,
The garden, in the presecreaty floughing I cleft.
Then, high I cargue to again.

AUFIDIUS:
A gone!
Yet please look to none, my lord, I pray not.
Here comes he comes the king my body!

CORIOLANUS:
He hath for his right murderers sun 


In [ ]:
with open('tiny-shakespeare-output.txt', 'w') as file:
  file.write(decode(model.generate(context, 10000)[0].tolist()))

# Encoder & Decoder Mechanism.

What we implemented is a Decoder only transformer.

Encoder & Cross-Attention:

In [ ]:
# ----------------ENCODER----------------|----------------DECODER----------------
# Les réseaux de neurones sont géniaux|<START>Neural networks are awesome<END> (<START> & <END> are special tokens)
# The Research Paper diagram is for Machine Translation.

# If you want to condition the generation based upon some additional information (like the French sentence, here).
# Encoder for French -> No triangular mask (They are free to talk with all) # Just encode the content of French Sentence.
# Decoder will get the keys and values from encoders (queries still from x)
# Every single block of the Decoder (Transformer) me feed karenge keys and values


# Condition the English Decoding not just because of what it knows but also the context of what it has fully seen for the French sentence.
# Our case we just have text and we want to to emmit it (So no conditional decoding/generation -> Hence, decoder only block)

# AK's Nano-GPT:

In [ ]:
# Instead of concatenating all the heads input (in Multi-Head Attention)
# We can also treat it like 4 D tensors (where the number of heads become one dimension).

# Gelu instead of Relu (Open AI) activation.

# Training Chat-GPT
# 2 phase ->
# 1. Pre-training phase: Similar to what we have done above (We train a decoder only transformer)
# And we basically train it on a big chunk of the internet (Trillions of tokens) (3 Billion tokens pe train -> GPT3). We can check the GPT Research Papers
# That will have all parameters, batch_size, head_size, layers, et cetera.
# But our model has per character level token/ Open AI/Other models -> Sub words-specific tokens (3/4th of a word -> 1 token)

# Now it just blabbers internet. If you ask it a question, it will try generating more further, right? Because that's what its trained for?
# Then how come Chat GPT? Unaligned???
# 2nd. stage to fine-tune it to align it be an assistant.
# Take 1000s of exaples/Not internet -> Questions and Answers.
# Fine tune GPT with Supervised Learning. (Explain A: Answer A)
# Reward model (Best option/answers/labeller)/ Reinforcement learning. -> Whole aligning/fine-tuning stage.
# Takes a model from a document completer -> QA agent. (Internal datasets) / Hard to replicate.